# Script para planilha de lista eleitoral

In [11]:
import pypdf
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   

from pathlib import Path
from functools import partial
from langchain_openai import  AzureChatOpenAI
from langchain.prompts import PromptTemplate
from utils_ccd import get_connection, extract_text_from_pdf, get_file_path

conn = get_connection()

In [2]:
df_lista = pd.read_excel("docs/lista_eleitoral_250527.xlsx")
processos = ','.join(map(lambda x: f"'{x}'", df_lista['NUM_PROCESSO']))

sql_processo = f''' 
SELECT *
FROM processo.dbo.Processos p
WHERE CONCAT(p.numero_processo, '/', p.ano_processo) IN ({processos});
'''
df_processo = pd.read_sql(sql_processo, conn)

sql_all_informacoes_processos = f'''
    SELECT concat(rtrim(inf.setor),'_',inf.numero_processo ,'_',inf.ano_processo,'_',RIGHT(concat('0000',inf.ordem),4),'.pdf') as arquivo,
    ppe.SequencialProcessoEvento as evento,
    CONCAT(inf.numero_processo,'/', inf.ano_processo) as processo,
    inf.*
    FROM processo.dbo.vw_ata_informacao inf INNER JOIN processo.dbo.Pro_ProcessoEvento ppe 
        ON inf.idinformacao = ppe.idinformacao
    WHERE concat(inf.numero_processo, '/', inf.ano_processo) in ({processos})
'''
all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)

def filter_infos(df):
    df = df.copy()
    df = df[~df['resumo'].str.contains('capa', case=False, na=False)]
    df = df[~df['resumo'].str.contains('digitalizado', case=False, na=False)]
    return df


/tmp/ipykernel_186959/1476759274.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_processo = pd.read_sql(sql_processo, conn)
/tmp/ipykernel_186959/1476759274.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)


In [22]:
llm = AzureChatOpenAI(model_name="gpt-4o")
def is_parecer_previo(texto):
    try:
        prompt = PromptTemplate.from_template("""
        Você é um agente que analisa e resume informações de processos do Tribunal de Contas do Estado do Rio Grande do Norte.

        O texto da informação é o seguinte:
        "{input}"
                                              
        Defina se a informação é um parecer prévio ou não.
        Responda apenas com "Sim" ou "Não". Não forneça explicações ou justificativas.                                           

        Sua resposta:
        """)

        chain = prompt | llm
        msg_obj = chain.invoke(texto)
        return msg_obj.content
    except Exception as e:
        print(f"Error processing {texto}: {e}")
        return None

In [3]:
processo = processos.split(',')[0].replace("'", "")
processo

'000270/2005'

In [4]:
info_processo = all_informacoes_processos[all_informacoes_processos['processo'] == processo]

In [5]:
info_processo = filter_infos(info_processo)

In [13]:
info_processo['caminho_arquivo'] = info_processo.apply(get_file_path, axis=1)

In [16]:
info_processo['caminho_arquivo']

2432    /media/informacoes_pdf/GCREN/GCREN_000270_2005...
2452    /media/informacoes_pdf/DAE_SEI/DAE_SEI_000270_...
Name: caminho_arquivo, dtype: object

In [18]:
info_processo['texto'] = info_processo['caminho_arquivo'].apply(extract_text_from_pdf)

In [20]:
print(info_processo['texto'].iloc[0])

 
 
 
 
 
 
 
 
PROCESSO Nº.: 270/2005-TC. 
ASSUNTO: DOCUMENTAÇÃO COMPROBATÓRIA DE DESPESA ANO 200 3- 
DILIGÊNCIA 3384-DAE. 
INTERESSADO: CÂMARA  MUNICIPAL DE SÃO PEDRO. 
CONSELHEIRO RELATOR: RENATO COSTA DIAS  
 
 
 
 
 
DESPACHO 
Natal – RN, 11/07/2018 
 
 
 
 
  
Acato o Despac ho da Diretoria de Atos e Execuções - DAE,  evento 
12, e tendo em vista a devida formalização do procedimento au tônomo de 
execução (Proc.  nº 6334/2018-TC),  determino o encaminhamento do presente 
processo ao Arquivo Geral desta Corte de Contas, para arquiv amento. 
 
 
 
 
RENATO COSTA DIAS 
Conselheiro-Relator 
Gabinete do Conselheiro Renato Costa Dias 
